## ASHRAE 140-2023

Section 8: CASE GC50b - Large Slab

_opensimula: v 0.6_

In [6]:
import opensimula as osm

caseGC50b_dict = {
    "name": "Case GC50b",
    "time_step": 3600,
    "n_time_steps": 2*8760,
    "initial_time": "01/01/2001 00:00:00",
    "simulation_file_met": "GC40b_met",
    "shadow_calculation": "NO",
    "components": [
        {
            "type": "File_met",
            "name": "GC40b_met",
            "file_type": "TMY2",
            "file_name": "../../mets/GC40b.TM2"
        },
        {
            "type": "Material",
            "name": "Insulation",
            "conductivity": 0.0001,
            "density": 1,
            "specific_heat": 1
        },
        {
            "type": "Material",
            "name": "Ground",
            "conductivity": 1.9,
            "density": 1490,
            "specific_heat": 1800
        },
        {
            "type": "Construction",
            "name": "Adiabatic_Wall",
            "solar_alpha": [0,0],
            "lw_epsilon": [0,0],
            "materials": [
                "Insulation"
            ],
            "thicknesses": [
                1.0
            ]
        },
        {
            "type": "Construction",
            "name": "Floor_Slab",
            "solar_alpha": [0,0],
            "lw_epsilon": [0,0],
            "materials": [
                "Ground"
            ],
            "thicknesses": [
                0.01
            ]
        },
        {
            "type": "Space_type",
            "name": "no_gain_space",
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "0",
            "infiltration": "0"
        },
        {
            "type": "Building",
            "name": "Building",
            "azimuth": 0,
            "initial_temperature": 10
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Building",
            "spaces_type": "no_gain_space",
            "floor_area": 6400,
            "volume": 17280,
            "furniture_weight": 0
        },
        {
            "type": "Building_surface",
            "name": "north_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "space_1",
            "ref_point": [
                80,
                80,
                0
            ],
            "width": 80,
            "height": 2.7,
            "azimuth": 180,
            "altitude": 0,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "Building_surface",
            "name": "east_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "space_1",
            "ref_point": [
                80,
                0,
                0
            ],
            "width": 80,
            "height": 2.7,
            "azimuth": 90,
            "altitude": 0,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "Building_surface",
            "name": "south_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "space_1",
            "ref_point": [
                0,
                0,
                0
            ],
            "width": 80,
            "height": 2.7,
            "azimuth": 0,
            "altitude": 0,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "Building_surface",
            "name": "west_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "space_1",
            "ref_point": [
                0,
                80,
                0
            ],
            "width": 80,
            "height": 2.7,
            "azimuth": -90,
            "altitude": 0,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "Building_surface",
            "name": "roof_wall",
            "construction": "Adiabatic_Wall",
            "spaces": "space_1",
            "ref_point": [
                0,
                0,
                2.7
            ],
            "width": 80,
            "height": 80,
            "azimuth": 0,
            "altitude": 90,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "Building_surface",
            "name": "floor_wall",
            "surface_type": "UNDERGROUND",
            "construction": "Floor_Slab",
            "ground_material": "Ground",
            "exterior_perimeter_wall_thickness": 0.24,
            "spaces": "space_1",
            "ref_point": [
                0,
                80,
                0
            ],
            "width": 80,
            "height": 80,
            "azimuth": 0,
            "altitude": -90,
            "h_cv": [
                100,
                100
            ]
        },
        {
            "type": "HVAC_perfect_system",
            "name": "system",
            "spaces": "space_1",
            "outdoor_air_flow": "0",
            "heating_setpoint": "30",
            "cooling_setpoint": "50",
            "system_on_off": "1"
        }
    ]
}


sim = osm.Simulation()
pro = sim.new_project("pro")
pro.read_dict(caseGC50b_dict)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Case GC50b
Checking completed.
Simulating Case GC50b: ...


100%|██████████| 17520/17520 [00:13<00:00, 1264.99step/s, n_iter=3]


In [7]:
suelo = pro.component("floor_wall")
t_suelo = suelo.variable("T_s0")  
t_dentro = suelo.variable("T_s1")
t_aire = pro.component("space_1").variable("temperature")
sim.plot(pro.dates(),[t_suelo, t_dentro, t_aire])


In [8]:

sim.plot(pro.dates(),[pro.component("system").variable("Q_sensible")])

In [9]:
import numpy as np

# Heating and Cooling Loads
q_cd = -pro.component("floor_wall").variable("q_cd1").values[24*365:]*0.001*80*80
Q_CD = q_cd.sum()  # kWh/year
q_heat = pro.component("system").variable("Q_sensible").values[24*365:]*0.001
Q_heat =q_heat.sum()
T_zone = pro.component("space_1").variable("temperature").values[24*365:].mean()
T_ground = pro.component("floor_wall").variable("T_s0").values[24*365:].mean()

peak_cd = q_cd.max()*1000
i_peak_cd = np.argmax(q_cd)
peak_heat = q_heat.max()*1000
i_peak_heat = np.argmax(q_heat)
date_peak_cd = pro.dates()[i_peak_cd]
date_peak_heat = pro.dates()[i_peak_heat]

T_ext = pro.component("GC40b_met").variable("temperature").values[24*365:]
T_ext_min= T_ext.min()
i_T_ext_min = np.argmin(T_ext)
date_T_ext_min = pro.dates()[i_T_ext_min]
n_min_values = (T_ext == T_ext_min).sum()
                
print(Q_CD, Q_heat, T_zone, T_ground)
print(peak_cd,date_peak_cd)
print(peak_heat,date_peak_heat)
print(T_ext_min,date_T_ext_min, n_min_values)
n_min_values

205699.31102887532 205826.5719875071 30.000000000000007 10.000639269406392
27339.154946860803 2001-02-02 14:30:00
27359.12741015386 2001-02-02 04:30:00
2.0 2001-01-08 03:30:00 15


np.int64(15)

In [5]:
print("U_ground: ",pro.component("floor_wall")._U_tot_) 
print("U_ground_dynamic: ",pro.component("floor_wall")._U_hat_ext_)

U_ground:  0.18345630582494712
U_ground_dynamic:  0.10128732372970638
